In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [4]:
hyper_epoch=50

In [6]:
thresholds={0: 0.05,\
            1:-0.45,\
            2:-0.85,\
            3:-1.65,\
            4:-1.65,\
            5:-0.75,\
            6:-1.65,\
            7:-1.95}

In [ ]:
for _ in range(hyper_epoch):
    # model parameters
    reuse = np.random.choice([True,False],p=[0.2,0.8])
    block = np.random.choice([NNConv_block,schnet_block],p=[0.6,0.4])
    head = np.random.choice([cat3Head_type,swapHead_type,cat3HeadPool_type,set2setHead_type,SimpleHead_type],\
                            p=[0.1875,0.1875,0.1875,0.1875,0.25])
    data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA.pickle',\
                             '../Data/{}_data_SOAP_expand_PCA.pickle',\
                             '../Data/{}_data_atomInfo.pickle'])
    batch_size = 32
    dim = int(np.random.choice([128,256,512]))
    epochs = 50
    clip = 2
    layer1 = int(np.random.choice([2,3]))
    layer2 = int(np.random.choice([2,3]))
    factor = int(np.random.choice([2,3]))
    lr = 1e-4
    epochs_type = epochs

    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [reuse,block,head,data,batch_size,dim,clip,\
                                              layer1,layer2,factor,epochs]])))

    for i in range(8):
        # load type data
        data_type = data.split('.pickle')[0]+ '_type_'+str(i)+'.pickle'
        train_dl,val_dl = get_data(data_type,batch_size)

        # init model
        model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
        paras = trainable_parameter(model)
        opt = Adam(paras,lr=lr)
        scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

        print('\nTraining type: {}'.format(i))
        model,train_loss_perType,val_loss_perType = train_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,\
                                                                    scheduler=scheduler,threshold=thresholds[i])
        
        if model is not None:
            save_results(train_loss_perType,val_loss_perType,reuse,block,\
                         head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')

            save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')